In [ ]:
import keras
import gymnasium as gym
import ale_py
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing

# Specific model file path
model_file = "/Users/rithwanali/Documents/Atari/Atari/models/spaceinvaders_qmodel_7549.keras"

# Register Atari environments
gym.register_envs(ale_py)

# Load the trained agent model
agent = keras.models.load_model(model_file)

# Create the Space Invaders environment
env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="human")
env = AtariPreprocessing(env)
env = FrameStack(env, 4)

# Reset the environment
state, _ = env.reset()

# Run the game
done = False
while not done:
    # First convert to a tensor for compute efficiency
    state_tensor = keras.ops.convert_to_tensor(state)
    
    # Shape of state is 4, 84, 84, but we need 84, 84, 4
    state_tensor = keras.ops.transpose(state_tensor, [1, 2, 0])
    
    # Add batch dimension
    state_tensor = keras.ops.expand_dims(state_tensor, 0)
    
    # Predict action probabilities
    action_probs = agent(state_tensor, training=False)
    
    # Select the best action
    action = keras.ops.argmax(action_probs[0]).numpy()
    
    # Take the action in the environment
    state, reward, done, _, _ = env.step(action)

# Close the environment when done
env.close()